In [ ]:
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./log.anon") }

fun add0WithinData(toFill: List<UData>): List<Pair<UInt, List<UData>>> {
    val filledQueryPerSecond = mutableListOf<Pair<UInt, List<UData>>>()
    var current = mutableListOf<UData>()
    var currentTime = toFill.first().unixTimeStamp
    toFill.forEach { u ->
        if (u.unixTimeStamp == currentTime) {
            current.add(u)
        } else {
            filledQueryPerSecond.add(Pair(currentTime, current))
            val afterFirst = currentTime + 1u
            val beforeLast = u.unixTimeStamp - 1u
            if (afterFirst == beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
            } else if (afterFirst < beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
                filledQueryPerSecond.add(Pair(beforeLast, listOf()))
            }
            current = mutableListOf(u)
            currentTime = u.unixTimeStamp
        }
    }
    filledQueryPerSecond.add(Pair(currentTime, current))
    return filledQueryPerSecond
}

val splitPerSuffix = data.groupBy { ud -> ud.domainName }.mapValues { (_, v) -> add0WithinData(v) }

val queryPerSecond = add0WithinData(data)

fun plotOverTime(
    udata: Map<List<String>, List<Pair<UInt, List<UData>>>>,
    title: String,
    includeSub: Boolean = true
): Plot {
    val perSuffixTraces = udata.map { (suffix, v) ->
        Trace() {
            val values = v.map { (s, udl) -> Pair(s, udl.filter { ud -> includeSub || ud.sub == false }) }
            name = suffix.joinToString(".")
            x.strings = values.map { (t, _) ->
                Instant.ofEpochSecond(t.toLong()).plus(Duration.ofHours(2)).toString() // Convert to CET
            }
            y.numbers = values.map { (_, udl) -> udl.size }
            type = TraceType.scatter
        }
    }.sortedByDescending { t -> t.y.numbers.sumOf { n -> n.toInt() } }

    return Plotly.plot {
        traces(perSuffixTraces)
        scatter {
            line {
                width = 1
            }
        }

        layout {
            width = 1700
            height = 950
            title {
                text = "$title"
            }
            xaxis {
                title {
                    text = "Time"
                }
            }
            yaxis {
                title {
                    text = "Amount of validations per second"
                }
            }
        }
    }
}


In [ ]:
plotOverTime(mapOf(Pair(listOf("all"), queryPerSecond)), "Amount of validations over time")

In [ ]:
val amountRootZone =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size == 1 }) }

plotOverTime(mapOf(Pair(listOf(), amountRootZone)), "Amount of validations over time for root zone")

In [ ]:
val amountTLDs =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }

plotOverTime(
    mapOf(Pair(listOf("TLDs and root zone"), amountTLDs)),
    "Amount of validations over time for TLDs and root zone",
    false
)

In [ ]:
plotOverTime(splitPerSuffix, "Amount of validations over time per suffix including subdomains")

In [ ]:
plotOverTime(
    splitPerSuffix,
    "Amount of validations over time per suffix excluding subdomains",
    false
)